In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB

In [76]:
import random
import csv

operations = [
    {
        "operation": "consultation derniers virements",
        "phrases": ["Consulte les derniers virements.", "Vérifie les virements récents.", "Regarde les dernières transactions."]
    },
    {
        "operation": "payer une facture",
        "phrases": ["Paye une facture d'<NATURE> de <MONTANT> euros.", "Effectue le paiement d'une facture d'<NATURE> d'un montant de <MONTANT> euros.", "Règle la facture d'<NATURE> de <MONTANT> euros."]
    },
    {
        "operation": "savoir le numéro de RIB",
        "phrases": ["Demande mon numéro de RIB.", "Obtiens mon numéro de RIB.", "Savoir mon RIB."]
    },
    {
        "operation": "demander une carte bancaire ou un carnet de chèque",
        "phrases": ["Demande une carte bancaire.", "Sollicite l'obtention d'une carte bancaire ou d'un carnet de chèque.", "Fais la demande pour obtenir une carte bancaire."]
    },
    {
        "operation": "consultation du solde",
        "phrases": ["Consulte mon solde.", "Vérifie le solde de mon compte.", "Regarde combien j'ai d'argent sur mon compte."]
    },
    {
        "operation": "virement",
        "phrases": ["Effectue un virement de <MONTANT> euros à <NOM>.", "Transfère <MONTANT> euros à <NOM>.", "Verse <MONTANT> euros à <NOM>."]
    }
]

destinataires = ["mohamed", "rachid", "khadija", "karima", "yassine", "karim", "omar", "fatima", "imane", "hicham"]
natures_facture = ["eau", "électricité", "téléphone"]

dataset = []

for operation in operations:
    for _ in range(5000):
        phrase_principale = random.choice(["Effectue", "Réalise", "Effectuer", "Faire"])
        phrase = random.choice(operation['phrases'])
        phrase = phrase.replace("<MONTANT>", str(random.randint(10, 1000)))
        montant = None
        destinataire = None
        nature_facture = None
        if operation['operation'] == 'virement':
            montant = random.randint(10, 1000)
            destinataire = random.choice(destinataires)
            phrase = phrase.replace("<NOM>", destinataire)
        elif operation['operation'] == 'payer une facture':
            montant = random.randint(10, 1000)
            nature_facture = random.choice(natures_facture)
            phrase = phrase.replace("<NATURE>", nature_facture)
        data = {
            "phrase": phrase,
            "operation": operation['operation'],
            "montant": montant,
            "destinataire": destinataire,
            "nature_facture": nature_facture
        }
        dataset.append(data)

with open('dataset_operations2.csv', 'w', newline='') as f:
    fieldnames = ['phrase', 'operation', 'montant', 'destinataire', 'nature_facture']
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(dataset)



In [77]:
data=pd.read_csv('C:/Users/HP/dataset_operations2.csv', encoding='ISO-8859-1')

In [79]:
data=data[['phrase','operation']]


In [80]:
data.head()

,phrase,operation
0,Vérifie les virements récents.,consultation derniers virements
1,Vérifie les virements récents.,consultation derniers virements
2,Vérifie les virements récents.,consultation derniers virements
3,Regarde les dernières transactions.,consultation derniers virements
4,Vérifie les virements récents.,consultation derniers virements


In [82]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


# Diviser les données en variables d'entrée (phrases) et variable cible (opération)
X = data['phrase']
y = data['operation']

# Vectorisation des phrases en utilisant TF-IDF
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)

# Entraîner un modèle de régression logistique
model = LogisticRegression()
model.fit(X_train, y_train)

# Évaluer la précision du modèle sur les données de test
accuracy = model.score(X_test, y_test)
print("Précision du modèle : {:.2f}%".format(accuracy * 100))

# Exemple de prédiction
phrase_test = "Consulte mon solde."
phrase_test_vectorized = vectorizer.transform([phrase_test])
prediction = model.predict(phrase_test_vectorized)
print("Phrase : {}".format(phrase_test))
print("Opération prédite : {}".format(prediction[0]))


Précision du modèle : 100.00%
Phrase : Consulte mon solde.
Opération prédite : consultation du solde


In [83]:
y_predict= model.predict(X_test)
print(classification_report(y_test,y_predict))

                                                    precision    recall  f1-score   support

                   consultation derniers virements       1.00      1.00      1.00       978
                             consultation du solde       1.00      1.00      1.00      1033
demander une carte bancaire ou un carnet de chèque       1.00      1.00      1.00       950
                                 payer une facture       1.00      1.00      1.00      1024
                           savoir le numéro de RIB       1.00      1.00      1.00      1046
                                          virement       1.00      1.00      1.00       969

                                          accuracy                           1.00      6000
                                         macro avg       1.00      1.00      1.00      6000
                                      weighted avg       1.00      1.00      1.00      6000

